In [ ]:
!echo Y | sudo apt-get install python3-opencv

In [ ]:
!pip install paho-mqtt

In [ ]:
import numpy as np
import cv2
from PIL import Image
import sys
import os
import urllib
import tensorflow.contrib.tensorrt as trt
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import tensorflow as tf
import numpy as np
import time
from tf_trt_models.detection import download_detection_model, build_detection_graph
import time
import paho.mqtt.client as mqtt
import base64
LOCAL_MQTT_HOST="mosquitto"
LOCAL_MQTT_PORT=1883
LOCAL_MQTT_TOPIC="test_topic"

def on_connect_local(client, userdata, flags, rc):
        print("connected to local broker with rc: " + str(rc))
        client.subscribe(LOCAL_MQTT_TOPIC)

def on_message(client,userdata, msg):
  try:
    print("message received!")
    # if we wanted to re-publish this message, something like this should work
    # msg = msg.payload
    # remote_mqttclient.publish(REMOTE_MQTT_TOPIC, payload=msg, qos=0, retain=False)
  except:
    print("Unexpected error:", sys.exc_info()[0])

local_mqttclient = mqtt.Client()
local_mqttclient.on_connect = on_connect_local
local_mqttclient.connect(LOCAL_MQTT_HOST, LOCAL_MQTT_PORT, 60)
local_mqttclient.on_message = on_message
# 1 should correspond to /dev/video1 , your USB camera. The 0 is reserved for the TX2 onboard camera
cap = cv2.VideoCapture(1)
count = 0
while(True):
    ret, frame = cap.read()
    #frame = cv2.imwrite("frame.png", frame)
    # We don't use the color information, so might as well save space
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    print( count == count+0)
    rc,png = cv2.imencode('.jpg', frame)

    #msg = png.tobytes()
    #with open('test.bin','a+') as f:
    #    f.write(msg)
    #cv2.imshow('cnts',sub_face)
    #cv2.waitKey(0)
    count = count +1
    with open('picture_out.jpg', 'wb') as f:
        f.write(png)
    IMAGE_PATH = 'picture_out.jpg'
    #time.sleep(1)
    image = Image.open(IMAGE_PATH)

    plt.imshow(image)

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    ax.imshow(image)

    plt.show()
    im = Image.open('picture_out.jpg')
    im_crop = im
    im_crop.save('picture_out'+str(count)+str(count)+'.jpg', quality=95)

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    ax.imshow(im_crop)

    plt.show()
    image = cv2.imread('picture_out'+str(count)+str(count)+'.jpg')
    #msg = image.tobytes()
    with open('picture_out'+str(count)+str(count)+'.jpg', "rb") as image:
        b64string = base64.b64encode(image.read()) 
    local_mqttclient.publish(LOCAL_MQTT_TOPIC,payload=b64string)
cap.release()